# Machine Learning - Overview

The purpose of this notebook is...

## Machine Learning Models Built:
- Ordinary Least Squares
- Stochastic Gradient Descent (SGD)
- ...

# Data Importing and Organization

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from mpl_toolkits.basemap import Basemap, cm
import seaborn as sns 

import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels

from statsmodels.graphics.gofplots import qqplot

from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
df = pd.read_csv('ML_dataset_filtered.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date').drop(['stid', 'year', 'day'], axis=1)

In [ ]:
df.head()

# Time Series Cross Validation

The time series energy data displays auto-correlation on a year-to-year scale. We will therefore use temporally contiguous blocks for cross-validation, forcing testing on more temporally distant records, reducing temporal dependence and reducing optimism in error estimates. We are attempting to ensure independence between cross-validation folds. 

https://onlinelibrary.wiley.com/doi/full/10.1111/ecog.02881


The data have been split into the following groups (years are inclusive):

- 1994-1995: Contiguous Fold 1
- 1996-1997: Contiguous Fold 2
- 1998-1999: Contiguous Fold 3
- 2000-2001: Contiguous Fold 4
- 2002-2003: Contiguous Fold 5
- 2004-2007: Test Data

The cross-validation strategy is as follows:
- Choose 1 contiguous fold as validation data and use remaining contiguous folds as training data
    - Calculate CV score
- Perform the above process 5 times, with each of the contiguous folds being the validation data
    - Calculate average CV score
- Finally, use all 5 contiguous folds as training data and calculate the test data score

The models will be evaluated based on the following:
- Average CV score
- Test data score

Hyperparameter Tuning:
- If required, hyperparameters can be tuned by calculating the average CV score on the 5 contiguous folds and choosing the optimal parameter

In [ ]:
def cv_score(reg, X_train, y_train, score_func=mean_absolute_error):
    '''
    DOCSTRING
    
    '''
    # In here for now to surpress a warning
    pd.options.mode.chained_assignment = None
    
    result = 0

    # Split data into 5 contiguous folds
    years = list(range(1994, 2004))
    n_fold = 5
    fold = 0
    
    cv_scores = pd.DataFrame()
    
    for i in list(range(0, len(years), 2)):
        
        X_tr = X_train[(X_train.index.year != years[i]) & (X_train.index.year != years[i+1])]
        y_tr = y_train[(y_train.index.year != years[i]) & (y_train.index.year != years[i+1])].to_frame()
        
        X_te = X_train.loc[str(years[i]):str(years[i+1])].astype(float)
        y_te = y_train.loc[str(years[i]):str(years[i+1])].to_frame().astype(float)
    
        if reg == 'ols':
            
            X_tr['energy'] = y_tr.loc[:, 'energy']
            
            ols_string = 'energy ~ ' + ''.join([i + ' + ' for i in X_tr.columns[:-1]])[:-3]

            m = ols(ols_string, X_tr).fit()
            mae = score_func(m.predict(X_te), y_te)
            result += mae
            
            temp_df = pd.DataFrame()
            temp_df['fold'] = [fold + 1]
            temp_df['cv_mae'] = [mae]
            cv_scores = cv_scores.append(temp_df)
        
        else:
            reg.fit(X_tr, y_tr)
            mae = score_func(reg.predict(X_te), y_te)
            result += mae
            
            temp_df = pd.DataFrame()
            temp_df['fold'] = [fold + 1]
            temp_df['cv_mae'] = [mae]
            cv_scores = cv_scores.append(temp_df)
            
        fold += 1
        
        average_cv = result / n_fold
        
    return average_cv, cv_scores

# Feature Selection

In [ ]:
# Extract only forecast hour 0 for the purposes of the below analysis
df_t0 = df.copy()

for col in df_t0.columns:
    if str(col)[-6:-1] == 'fhour':
        if str(col)[-1] != '0':
            del df_t0[col]
            
# Compute the correlation matrix
corr = df_t0.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set column labels
column_labels = ['Energy availability', 'Latitude', 'Longitude', 'Elevation','3-Hour accumulated\nprecipitation', 'Downward long-wave\nradiative flux average','Downward short-wave\nradiative flux average', 'Air pressure at\nmean sea level','Precipitable water', 'Specific humidity','Total cloud cover', 'Total column-integrated\ncondensate ','Maximum temperature\nover the past 3 hours', 'Minimum temperature\nover the past 3 hours','Current temperature\nat 2m above ground', ' Surface temperature','Surface upward\nlong-wave radiation', 'Atmosphere upward\nlong-wave radiation','Surface upward\nshort-wave radiation', 'Month']

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Draw the heatmap with the mask
sns.heatmap(corr, mask=mask, cmap='coolwarm', vmin=-1, vmax=1, center=0,
            square=True, linewidths=.5, xticklabels=column_labels, yticklabels=column_labels)

plt.show()

In [ ]:
corr[corr > 0.9]

dlwrf_sfcfhour0: 0.279714
- pwat_eatmfhour0
- spfh_2m_lfhour0
- tmax_2m_lfhour0
- tmin_2m_lfhour0
- tmp_2m_lafhour0
- tmp_sfc_lfhour0
- ulwrf_sfcfhour0

pwat_eatmfhour0: 0.201308
- dlwrf_sfcfhour0
- spfh_2m_lfhour0 
- tmin_2m_lfhour0
- tmp_2m_lafhour0
- tmp_sfc_lfhour0
- ulwrf_sfcfhour0

spfh_2m_lfhour0: 0.344125
- dlwrf_sfcfhour0
- pwat_eatmfhour0
- tmax_2m_lfhour0
- tmin_2m_lfhour0
- tmp_2m_lafhour0
- tmp_sfc_lfhour0
- ulwrf_sfcfhour0

tcdc_eatmfhour0: -0.343932	(keep)
- tcolc_eatfhour0

tcolc_eatfhour0: -0.343467
- tcdc_eatmfhour0

tmax_2m_lfhour0: 0.509588 (keep)
- dlwrf_sfcfhour0
- spfh_2m_lfhour0

tmin_2m_lfhour0: 0.480312
- dlwrf_sfcfhour0
- pwat_eatmfhour0
- spfh_2m_lfhour0
- tmax_2m_lfhour0
- tmp_2m_lafhour0
- tmp_sfc_lfhour0
- ulwrf_sfcfhour0

tmp_2m_lafhour0: 0.479993
- dlwrf_sfcfhour0
- pwat_eatmfhour0
- spfh_2m_lfhour0
- tmax_2m_lfhour0
- tmin_2m_lfhour0
- tmp_sfc_lfhour0
- ulwrf_sfcfhour0

tmp_sfc_lfhour0: 0.464945
- dlwrf_sfcfhour0
- pwat_eatmfhour0
- spfh_2m_lfhour0
- tmax_2m_lfhour0
- tmin_2m_lfhour0
- tmp_2m_lafhour0
- ulwrf_sfcfhour0

ulwrf_sfcfhour0: 0.485866 (keep)
- dlwrf_sfcfhour0
- pwat_eatmfhour0
- spfh_2m_lfhour0
- tmax_2m_lfhour0
- tmin_2m_lfhour0
- tmp_2m_lafhour0
- tmp_sfc_lfhour0

In [ ]:
plt.subplots(figsize=(12,12))
plt.scatter(df['spfh_2m_lfhour0'], df['pwat_eatmfhour0'], s=0.1, alpha=0.5)
plt.show()

# Model: OLS - BASELINE (NO FEATURE SELECTION)

In [ ]:
X_train = df.loc['1994':'2003'].iloc[:, 1:]
y_train = df.energy.loc['1994':'2003']
X_test = df.loc['2004':'2007'].iloc[:, 1:]
y_test = df.energy.loc['2004':'2007']

In [ ]:
df_ols = df.copy()
df_ols.head()

In [ ]:
average_cv, cv_scores = cv_score('ols', X_train, y_train, score_func=mean_absolute_error)
print('Average CV MAE: %.0f' % average_cv)
cv_scores.reset_index(drop=True)

In [ ]:
ols_string = 'energy ~ ' + ''.join([i + ' + ' for i in df_ols.columns[1:]])[:-3]

m_ols = ols(ols_string,df_ols).fit()
print('MAE: %.0f' % mean_absolute_error(m_ols.predict(X_test), y_test))
print(m_ols.summary())

# Model: OLS - FORECAST HOUR0

In [ ]:
df_ols_t0 = df_t0.copy()

X_train = df_ols_t0.loc['1994':'2003'].iloc[:, 1:]
y_train = df_ols_t0.energy.loc['1994':'2003']
X_test = df_ols_t0.loc['2004':'2007'].iloc[:, 1:]
y_test = df_ols_t0.energy.loc['2004':'2007']

In [ ]:
average_cv, cv_scores = cv_score('ols', X_train, y_train, score_func=mean_absolute_error)
print('Average CV MAE: %.0f' % average_cv)
cv_scores.reset_index(drop=True)

In [ ]:
ols_string = 'energy ~ ' + ''.join([i + ' + ' for i in df_ols_t0.columns[1:]])[:-3]

m_ols = ols(ols_string,df_ols_t0).fit()
print('MAE: %.0f' % mean_absolute_error(m_ols.predict(X_test), y_test))
print(m_ols.summary())

# Model: OLS - REDUCED FEATURES

In [ ]:
df_ols_fs = df.copy()
del df_ols_fs['month']
del df_ols_fs['nlat']
del df_ols_fs['elon']
del df_ols_fs['elev']
del df_ols_fs['dlwrf_sfcfhour0']
del df_ols_fs['dlwrf_sfcfhour1']
del df_ols_fs['dlwrf_sfcfhour2']
del df_ols_fs['dlwrf_sfcfhour3']
del df_ols_fs['dlwrf_sfcfhour4']
del df_ols_fs['pwat_eatmfhour0']
del df_ols_fs['pwat_eatmfhour1']
del df_ols_fs['pwat_eatmfhour2']
del df_ols_fs['pwat_eatmfhour3']
del df_ols_fs['pwat_eatmfhour4']
del df_ols_fs['spfh_2m_lfhour0']
del df_ols_fs['spfh_2m_lfhour1']
del df_ols_fs['spfh_2m_lfhour2']
del df_ols_fs['spfh_2m_lfhour3']
del df_ols_fs['spfh_2m_lfhour4']
del df_ols_fs['tcolc_eatfhour0']
del df_ols_fs['tcolc_eatfhour1']
del df_ols_fs['tcolc_eatfhour2']
del df_ols_fs['tcolc_eatfhour3']
del df_ols_fs['tcolc_eatfhour4']
del df_ols_fs['tmin_2m_lfhour0']
del df_ols_fs['tmin_2m_lfhour1']
del df_ols_fs['tmin_2m_lfhour2']
del df_ols_fs['tmin_2m_lfhour3']
del df_ols_fs['tmin_2m_lfhour4']
del df_ols_fs['tmp_2m_lafhour0']
del df_ols_fs['tmp_2m_lafhour1']
del df_ols_fs['tmp_2m_lafhour2']
del df_ols_fs['tmp_2m_lafhour3']
del df_ols_fs['tmp_2m_lafhour4']
del df_ols_fs['tmp_sfc_lfhour0']
del df_ols_fs['tmp_sfc_lfhour1']
del df_ols_fs['tmp_sfc_lfhour2']
del df_ols_fs['tmp_sfc_lfhour3']
del df_ols_fs['tmp_sfc_lfhour4']
df_ols_fs.head()

In [ ]:
X_train = df_ols_fs.loc['1994':'2003'].iloc[:, 1:]
y_train = df_ols_fs.energy.loc['1994':'2003']
X_test = df_ols_fs.loc['2004':'2007'].iloc[:, 1:]
y_test = df_ols_fs.energy.loc['2004':'2007']

In [ ]:
average_cv, cv_scores = cv_score('ols', X_train, y_train, score_func=mean_absolute_error)
print('Average CV MAE: %.0f' % average_cv)
cv_scores.reset_index(drop=True)

In [ ]:
ols_string = 'energy ~ ' + ''.join([i + ' + ' for i in df_ols_fs.columns[1:]])[:-3]
m_ols_fs = ols(ols_string,df_ols_fs).fit()
print('MAE: %.0f' % mean_absolute_error(m_ols_fs.predict(X_test), y_test))
print(m_ols_fs.summary())

# Model: SGD - NO FEATURE SELECTION

In [ ]:
X_train = df.loc['1994':'2003'].iloc[:, 1:]
y_train = df.energy.loc['1994':'2003']
X_test = df.loc['2004':'2007'].iloc[:, 1:]
y_test = df.energy.loc['2004':'2007']

In [ ]:
scaler = StandardScaler()
m = SGDRegressor(max_iter=1000, alpha=.00001, tol=.001)
reg = make_pipeline(scaler, m)

In [ ]:
average_cv, cv_scores = cv_score(reg, X_train, y_train, score_func=mean_absolute_error)
print('Average CV MAE: %.0f' % average_cv)
cv_scores.reset_index(drop=True)

In [ ]:
scaler = StandardScaler()
reg = SGDRegressor(max_iter=1000, alpha=.00001, tol=.001)
pipeline = make_pipeline(scaler, reg)

pipeline.fit(X_train, y_train)
print('MAE: %.0f' % mean_absolute_error(pipeline.predict(X_test), y_test))

# Model: GRADIENT BOOSTING REGRESSOR
- https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html

In [ ]:
X_train = df.loc['1994':'2003'].iloc[:, 1:].reset_index()
y_train = df.energy.loc['1994':'2003'].reset_index()
X_test = df.loc['2004':'2007'].iloc[:, 1:].reset_index()
y_test = df.energy.loc['2004':'2007'].reset_index()

In [ ]:
def gbr_gridsearch(X_train, y_train, n_estimators_space, max_features_space, max_depth_space):
    
    years = list(range(1994, 2004))

    train_indices =  [X_train[(X_train.Date.dt.year != years[i]) & (X_train.Date.dt.year != years[i+1])].index for i in list(range(0, len(years), 2))]
    test_indices = [y_train[(y_train.Date.dt.year != years[i]) & (y_train.Date.dt.year != years[i+1])].index for i in list(range(0, len(years), 2))]
    custom_cv = zip(train_indices, test_indices)
    
    param_grid = {'n_estimators': n_estimators_space, 'max_features': max_features_space, 'max_depth':max_depth_space}
    reg = GradientBoostingRegressor(loss='lad', subsample=0.5)
    reg_cv = GridSearchCV(reg, param_grid, cv=custom_cv, verbose=50, n_jobs=2)
    
    reg_cv.fit(X_train.iloc[:,1:], y_train.iloc[:,1:].values.ravel())
    
    print("Tuned Gradient Boosting Parameters: {}".format(reg_cv.best_params_)) 
    print("Best score is {}".format(reg_cv.best_score_))
    
    return reg_cv

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

n_estimators_space = np.asarray([500, 1000, 1500])
max_features_space = np.asarray([6, 10, 14])
max_depth_space = np.asarray([3, 4, 5, 6, 7, 8, 9])
gbr_reg_cv = gbr_gridsearch(X_train, y_train, n_estimators_space, max_features_space, max_depth_space)

In [ ]:
X_train = df.loc['1994':'2003'].iloc[:, 1:]
y_train = df.energy.loc['1994':'2003']
X_test = df.loc['2004':'2007'].iloc[:, 1:]
y_test = df.energy.loc['2004':'2007']

In [ ]:
reg = GradientBoostingRegressor(loss='lad', n_estimators=1000, max_features=10, max_depth=7, subsample=0.5, verbose=1)

In [ ]:
reg.fit(X_train, y_train)

In [ ]:
mae = mean_absolute_error(reg.predict(X_test), y_test)
mae

In [ ]:
reg.score(X_test, y_test)

# Model Evaluation

## Fitted vs Actual Scatterplot

In [ ]:
y_test = df.energy.loc['2004':'2007']
plt.subplots(figsize=(12, 12))
plt.scatter(m_ols.predict(X_test), y_test, s=0.1, alpha=0.5)
plt.plot([0,35000000], [0,35000000], '-', c='red')
plt.xlabel("Fitted Value")
plt.ylabel("Actual Energy")
plt.title("Relationship between Fitted Values and Actual Energy")
plt.show()

## Fitted vs Actual Histogram

In [ ]:
X_test

In [ ]:
X_test = df_ols_t0.loc['2004':'2007'].iloc[:, 1:]

plt.subplots(figsize=(12,12))
plt.hist(m_ols.predict(X_test), bins=100, color='blue', alpha=0.5)
plt.hist(y_test, bins=100, color='red', alpha=0.5)
plt.title("Histogram of Fitted and Actual Energy")
plt.xlabel("Energy")
plt.ylabel("Frequency")
plt.legend(['Fitted Values', 'Actual Values'])
plt.show()

## Fitted Values vs Residuals

In [ ]:
plt.subplots(figsize=(12,12))
plt.scatter(m_ols.predict(X_test), m_ols.resid, s=0.01, alpha=0.5)
plt.xlabel("Predicted Value")
plt.ylabel("Residuals")
plt.title("Relationship between Fitted Values and Residuals")

plt.show()

In [ ]:
_, _, f_value, f_pvalue = statsmodels.stats.diagnostic.het_breuschpagan(m_ols.resid, df_ols.iloc[:,1:])

In [ ]:
print('bp test f_value: ' + str(f_value))
print('bp test f_pvalue: ' + str(f_pvalue))

- the very low p-value indicates a very likely violation of homoscedasticity (constant variance)

## Histogram of Residuals

In [ ]:
plt.subplots(figsize=(12,12))
plt.hist(m_ols_filt.resid, bins=50)
plt.title("Histogram of Residuals")
plt.xlabel("Residual")
plt.ylabel("Frequency")
plt.show()

In [ ]:
statsmodels.stats.diagnostic.normal_ad(m_ols.resid)

- The Anderson-Darling Normality test has the null hypothesis that the data is normally distributed. The p-value of the test statistic is very small, thus we reject the null hypothesis that the data is normally distributed.

## Quantile-Quantile Plots

In [ ]:
qqplot(m_ols.resid, line='45', fit=True)

## Mean Absolute Error

In [ ]:
# MAE training data m_ols_t0
statsmodels.tools.eval_measures.meanabs(df_ols_fs.energy, m_ols_t0.fittedvalues)

In [ ]:
# MAE training data m_ols
statsmodels.tools.eval_measures.meanabs(df_ols_fs.energy, m_ols.fittedvalues)

In [ ]:
# MAE training data m_ols_fs
statsmodels.tools.eval_measures.meanabs(df_ols_fs.energy, m_ols_fs.fittedvalues)

In [ ]:
# MAE training data m_ols_filt
statsmodels.tools.eval_measures.meanabs(df_ols_filt.energy, m_ols_filt.fittedvalues)

In [ ]:
mae / df_ols_filt.energy.mean() * 100